In [0]:
import unicodedata
import re
import keras
import tensorflow as tf
import numpy as np


import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import pandas as pd
import re
import unicodedata
import urllib3
import shutil
import zipfile
import itertools


from keras.layers import Embedding, LSTM, Dense, Activation, Dot, concatenate, TimeDistributed

import keras
import tensorflow as tf
from keras import preprocessing

tf.random.set_seed(1234)


Using TensorFlow backend.


Vamos a armar un traductor del ingles al español usando un modelo seq2seq con atencion. Para eso primero necesitamos pares de frases en inglés y español

In [0]:
#!wget http://www.manythings.org/anki/spa-eng.zip
#!unzip spa-eng.zip

In [0]:
df = pd.read_csv("./spa.txt", sep = "\t",header=None)

In [0]:
esp_data_raw = df.iloc[:,1].tolist()
eng_data_raw = df.iloc[:,0].tolist()

In [0]:
## defino funciones de limpieza
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
    
def normalize_string(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

In [0]:
## tengo la data limpia y le agrego tokens de inicio y de fin
raw_data_en = [normalize_string(data) for data in eng_data_raw]
raw_data_es_in = ['<start> ' + normalize_string(data) +" <end>" for data in esp_data_raw]
raw_data_es_out = [normalize_string(data) +" <end>" for data in esp_data_raw]

In [0]:
## defino el largo maximo de la frase
max_length = 50

In [0]:
## instancio el tokenizador para ingles. Tokenizo y hago padding
en_tokenizer = preprocessing.text.Tokenizer(filters="")
en_tokenizer.fit_on_texts(raw_data_en)
data_en = en_tokenizer.texts_to_sequences(raw_data_en)
data_en = preprocessing.sequence.pad_sequences(data_en, padding='post',maxlen= max_length)

In [0]:
## repito en español. Voy a tener uno para el input y uno para
## usar en el teacher forcing

es_tokenizer = preprocessing.text.Tokenizer(filters='')
es_tokenizer.fit_on_texts(raw_data_es_in)
es_tokenizer.fit_on_texts(raw_data_es_out)

data_es_in = es_tokenizer.texts_to_sequences(raw_data_es_in)
data_es_in = preprocessing.sequence.pad_sequences(data_es_in, padding='post',maxlen= max_length)

data_es_out = es_tokenizer.texts_to_sequences(raw_data_es_out)
data_es_out = preprocessing.sequence.pad_sequences(data_es_out, padding='post',maxlen= max_length)

In [0]:
## defino el tamaño del vocabulario (le sumo 1 por el 0 del padding)
max_words_es = len(es_tokenizer.index_word.keys())+1
max_words_en = len(en_tokenizer.index_word.keys())+1

## y algunas caracteristicas de la red
embedding_dim= 1024
lstm_size = 1024


In [0]:
## voy a usar una sola capa de lstm con dropout. En el paper de luong usan un
## stack de 4. Pero ellos tenian aprox 4 millones de frases y entrenaban 7 dias

encoder_input = keras.Input(shape=(max_length,))
decoder_input = keras.Input(shape=(max_length,))

encoder_embedding = Embedding(max_words_en, embedding_dim, input_length = max_length,mask_zero=True)
encoder = LSTM(lstm_size, return_state=True,return_sequences = True, dropout=0.2)
decoder_embedding = Embedding(max_words_es, embedding_dim, input_length = max_length,mask_zero=True)
decoder = LSTM(lstm_size,return_sequences=True, return_state=True,dropout=0.2)

e_embedding = encoder_embedding(encoder_input)
encoder_out,state_h,state_c = encoder(e_embedding)
encoder_states = [state_h, state_c]

d_embedding = decoder_embedding(decoder_input)
decoder_out,_,_ = decoder(d_embedding, initial_state=encoder_states)

## si no quiero usar atencion puedo cortar acá y enchufar ya la salida:
#output = TimeDistributed(Dense(max_words_es, activation="softmax"))(decoder_out)


attention = Dot(axes=(2, 2))([decoder_out, encoder_out])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

 
context = Dot(axes=(2,1))([attention, encoder_out])
print('context', context)

decoder_mas_context = concatenate([context, decoder_out])
print('decoder_combined_context', decoder_mas_context)

output = TimeDistributed(Dense(64, activation="tanh"))(decoder_mas_context)
output = TimeDistributed(Dense(max_words_es, activation="softmax"))(output)
print('output', output)


model = keras.Model(inputs=[encoder_input,decoder_input], outputs=output)

attention Tensor("attention/truediv:0", shape=(None, 50, 50), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(None, 50, 1024), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(None, 50, 2048), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(None, 50, 25238), dtype=float32)


In [0]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
    clipnorm = 5
)

In [0]:
model.compile(optimizer = adam, loss="sparse_categorical_crossentropy")
history = model.fit([data_en,data_es_in], np.expand_dims(data_es_out,2), batch_size=128, epochs=10, validation_split=.2, shuffle = True)

Train on 99016 samples, validate on 24754 samples
Epoch 1/10
99016/99016 [==============================] - 857s 9ms/step - loss: 0.8387 - val_loss: 1.3560
Epoch 2/10
99016/99016 [==============================] - 856s 9ms/step - loss: 0.6208 - val_loss: 1.0911
Epoch 3/10
99016/99016 [==============================] - 855s 9ms/step - loss: 0.4502 - val_loss: 0.8905
Epoch 4/10
99016/99016 [==============================] - 852s 9ms/step - loss: 0.3392 - val_loss: 0.7663
Epoch 5/10
99016/99016 [==============================] - 855s 9ms/step - loss: 0.2634 - val_loss: 0.6957
Epoch 6/10
99016/99016 [==============================] - 853s 9ms/step - loss: 0.2129 - val_loss: 0.6576
Epoch 7/10
99016/99016 [==============================] - 855s 9ms/step - loss: 0.1771 - val_loss: 0.6400
Epoch 8/10
99016/99016 [==============================] - 852s 9ms/step - loss: 0.1504 - val_loss: 0.6371
Epoch 9/10
99016/99016 [==============================] - 852s 9ms/step - loss: 0.1294 - val_loss: 0.6

In [0]:
## me armo una funcion para traducir usando el modelo.
def traducir(frase,palabras=False):
  infer_deco_input = np.zeros(max_length)
  infer_deco_input[0] = es_tokenizer.word_index["<start>"]
  pos =0
  en_input = frase.reshape(1,-1)
  es_input= infer_deco_input.reshape(1,-1)
  while pos <= max_length:
    pred= model.predict([en_input,es_input]).argmax(2)
    es_input[0,pos+1] = pred[0][pos]
    if pred[0][pos] in (0,1):
      break
    pos +=1
  traduccion = es_input

  if palabras:
    texto= " ".join([es_tokenizer.index_word[x] for x in traduccion[0] if x !=0])
    return traduccion,texto

  return traduccion

In [0]:
## si falla entrenar 2 mas
"i need chocolate" in eng_data_raw

False

In [0]:
frase_a_traducir = "i need chocolate"
prueba_predict_en = [en_tokenizer.word_index[x] for x in frase_a_traducir.split()]
prueba_predict_en = preprocessing.sequence.pad_sequences([prueba_predict_en], padding='post',maxlen= max_length)[0]

In [0]:
print(" ".join([en_tokenizer.index_word[x] for x in prueba_predict_en if x!= 0]))
print(traducir(prueba_predict_en,True)[1])

i need chocolate
<start> necesito chocolate . <end>
